In [5]:
from bs4 import BeautifulSoup
import requests 

import pandas as pd

In [6]:
response = requests.get(url="https://en.wikipedia.org/wiki/User:Michael_J/County_table")

In [7]:
response.status_code

200

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
table = soup.find_all('table')[0]

In [92]:
FIPS = [] # column 2
county = [] # column 3
population = [] # column 5
area = [] # column 10, total area in km^2
latitude = [] # column 12
longitude = [] # column 13

In [93]:
for tr in table.find_all('tr')[1:]:
    # FIPS identifier
    tr_fips = tr.find('td').next_sibling.next_sibling.next_sibling.next_sibling
    FIPS.append(tr_fips.text)

    # County name
    tr_county = tr_fips.next_sibling.next_sibling
    county.append(tr_county.text)

    # Population
    tr_population = tr_county.next_sibling.next_sibling.next_sibling.next_sibling
    population.append(int(''.join(tr_population.text.split(','))))

    # Total area in km^2
    tr_area = tr_population.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
    area.append(float(''.join(tr_area.text.split(','))))

    # Latitude and longitude
    tr_latitude = tr_area.next_sibling.next_sibling.next_sibling.next_sibling
    latitude.append(float(tr_latitude.text.strip().strip('°').strip('+').strip('\n')))
    tr_longitude = tr_latitude.next_sibling.next_sibling
    longitude.append(float(tr_longitude.text.strip().strip('°').strip('+').strip('\n').replace('–','-')))

In [97]:
df = pd.DataFrame(data={'FIPS': FIPS, 'County': county, 'Population': population, 'Area (km^2)': area, 'Latitude': latitude, 'Longitude': longitude})

In [101]:
df.to_csv('county_census_data.csv', index=False)